<Advanced sensor validation using optimization.>

## Import Library

In [1]:
import joblib
import numpy as np
import pandas as pd

## Load Data

In [2]:
test_data = pd.read_csv('test_data.csv', index_col='DateTime', encoding='utf-8-sig')
test_x = test_data.values[:, :-1]
test_y = test_data.values[:, -1]
tag_list = test_data.columns[:-1]
x_num = len(tag_list)

In [3]:
test_data

,GAS_F,WTR_F,WTR_T,STM_T,STM_P,FLUE_T2,FLUE_O2
DateTime,,,,,,,
1384,1637.32,21.2,97.6,216.5,20.3,159,2.894


In [4]:
x_data = test_x[0].reshape(-1, x_num)
y_data = test_y[0]
y_fault_reference = 0.05

In [5]:
msg = {"unit_name": "Test Unit"}
msg['step_name'] = "Test Step"

# Algo 1. Diagnosis Fault and Reconstruct Using PLS Model

In [6]:
pca_info = joblib.load('pca_info.pkl')
pls_model = joblib.load('pls_model.pkl')
pls_info = joblib.load('pls_info.pkl')

In [7]:
from FaultDetect.algorithms import diagnose_fault

In [8]:
detect_msg_pca, identify_msg_pca, fault_variable_pca, detect_msg_vs, y_predict = diagnose_fault(x=x_data, y_measure=y_data,
                                                                                                y_predict_model=pls_model,
                                                                                                model_fault_reference=y_fault_reference,
                                                                                                pca_model_info=pca_info, pca_limit=pca_info['limit'],
                                                                                                tag_list=tag_list, 
                                                                                                msg=msg,
                                                                                                group_reference=None)
detect_msg_pca, identify_msg_pca, fault_variable_pca, detect_msg_vs, y_predict

Reconstruct X: [1637.32] ---> [1346.10689139]
Y Error- Y Predict:2.944, Y Measured:2.894


('Test Unit > Test Step > Fault Detect',
 'Test Unit > Test Step > GAS_F Error',
 'GAS_F',
 'Test Unit > Test Step > Y Error',
 2.9440021661744207)

# Algo 2. Diagnosis Fault Using PLS Model

In [9]:
pls_model = joblib.load('pls_model.pkl')
pls_info = joblib.load('pls_info.pkl')

In [10]:
from FaultDetect.algorithms import diagnose_fault_using_virtual_sensors

In [11]:
y_predict, detect_alarm, alarm_message, fault_variable = diagnose_fault_using_virtual_sensors(x=x_data,
                                                                                              y_measure=y_data,
                                                                                              y_predict_model=pls_model, 
                                                                                              model_fault_reference=y_fault_reference,
                                                                                              model_info=pls_info,
                                                                                              tag_list=tag_list,
                                                                                              msg=msg)
y_predict, detect_alarm, alarm_message, fault_variable

(2.823457885517407, True, 'Test Unit > Test Step > GAS_F is Fault', 'GAS_F')

# Algo 3. Diagnosis Fault Using PCA Model
1. Fault Detect
2. Fault Identfication

In [12]:
pca_info = joblib.load('pca_info.pkl')

In [13]:
from FaultDetect.algorithms import diagnose_fault_using_pca_model

In [14]:
detect_alarm, alarm_messsage, fault_variable = diagnose_fault_using_pca_model(x=x_data, 
                                                                              model_info=pca_info, 
                                                                              model_fault_reference=pca_info['limit'],
                                                                              tag_list=tag_list, msg=msg)
print(detect_alarm, alarm_messsage, fault_variable)

True Test Unit > Test Step > GAS_F is Fault GAS_F


# Algo 4. 

In [15]:
pls_model = joblib.load('pls_model.pkl')

In [16]:
from FaultDetect.algorithms import predict_y_using_virtual_sensors

In [17]:
detect_alarm, detect_msg, y_predict = predict_y_using_virtual_sensors(x=x_data,
                                                                      y_predict_model=pls_model,
                                                                      model_fault_reference=y_fault_reference,
                                                                      msg=msg,
                                                                      y_measure=y_data)
detect_alarm, detect_msg, y_predict

(True, 'Test Unit > Test Step > Error', 2.823457885517407)